Cleaning the dataset is the first part of the Tardis project. 
We have to select which values are mandatory, which must be corrected and which must be ignored

In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The clean_average function handles the average column. It returns a new dataframe with the column specified according to the following rules
    - N/A or <null> value -> 0.0
    - Values formated to HOUR/MIN/SEC (as a string)

In [238]:
def clean_average(column, dataframe):
    df = dataframe.copy()
    
    #possibly add verif for aberrant values
    df[column] = df[column].fillna(0.0)
    total_seconds = (df[column] * 60).astype(int)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    df[column] = hours.astype(str) + '.' + minutes.astype(str) + '.' + seconds.astype(str)
    return df

The clean_Pct function handles the Pct column. It returns a new dataframe with the column specified according to the following rules
    - N/A or <null> value -> 0.0
    - Values formated to HOUR/MIN/SEC (as a string)

In [239]:
def clean_Pct(column, dataframe):
    df = dataframe.copy()
    
    #possibly add verif for aberrant values (like > 1560)
    df[column] = df[column].fillna(0.0)
    total_seconds = (df[column] * 60).astype(int)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    df[column] = hours.astype(str) + '.' + minutes.astype(str) + '.' + seconds.astype(str)
    return df

The handle_column_with_train_number handles the columns that contain a number of train. It returns a new dataframe with the column specified according to the following rules:
    - replace non numeric values or float values to null
    - check if the values is < to the number of train at departure (because it is not possible to have more delayed trains then trains at departure)

In [240]:
def handle_column_with_train_number(column, dataframe):
    df = dataframe.copy()

    df[column] = df[column].replace(['N/A', ''], np.nan)
    scheduled = pd.to_numeric(df['Number of scheduled trains'].replace(['N/A', ''], np.nan), errors='coerce')
    df[column] = df[column].where(df[column] <= scheduled, np.nan)
    df[column] = pd.to_numeric(df[column], errors='coerce')
    df[column] = df[column].where(df[column].apply(lambda x: pd.isna(x) or x.is_integer()))
    df[column] = df[column].fillna(0).astype(int)
    return df


The tab containing all the destinations for spelling correction and the diff function to do it

In [241]:
all_city = [
    "PARIS MONTPARNASSE",
    "QUIMPER",
    "ST MALO",
    "ST PIERRE DES CORPS",
    "STRASBOURG",
    "PARIS NORD",
    "LYON PART DIEU",
    "TOURCOING",
    "NANTES",
    "PARIS VAUGIRARD",
    "BORDEAUX ST JEAN",
    "PARIS LYON",
    "MARNE LA VALLEE",
    "CHAMBERY CHALLES LES EAUX",
    "MARSEILLE ST CHARLES",
    "FRANCFORT",
    "ZURICH",
    "ANGOULEME",
    "POITIERS",
    "TOURS",
    "METZ",
    "REIMS",
    "PARIS EST",
    "DOUAI",
    "MULHOUSE VILLE",
    "VALENCE ALIXAN TGV",
    "STUTTGART",
    "BARCELONA",
    "ANGERS SAINT LAUD",
    "LAVAL",
    "NANCY",
    "LILLE",
    "GRENOBLE",
    "LE CREUSOT MONTCEAU MONTCHANIN",
    "MACON LOCHE",
    "NIMES",
    "ITALIE",
    "RENNES",
    "BREST",
    "LA ROCHELLE VILLE",
    "LE MANS",
    "VANNES",
    "DUNKERQUE",
    "AVIGNON TGV",
    "BELLEGARDE (AIN)",
    "BESANCON FRANCHE COMTE TGV",
    "DIJON VILLE",
    "MONTPELLIER",
    "MADRID",
    "ARRAS",
    "AIX EN PROVENCE TGV",
    "ANNECY",
    "NICE VILLE",
    "SAINT ETIENNE CHATEAUCREUX",
    "TOULON",
    "GENEVE",
    "PERPIGNAN",
    "LAUSANNE",
    "TOULOUSE MATABIAU"
]

def diff(string, to_cmp):
    nb_error = 0
    for i in range(len(string)):
        if string[i] != to_cmp[i]:
            nb_error += 1
    return nb_error

We start by reading he csv file and removing duplicates values

In [242]:
df = pd.read_csv('dataset.csv', delimiter=';', on_bad_lines='warn')
df = df.drop_duplicates()
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

We clean the 'Date' column by applying a YYYY/MM format, correcting small mistakes and removing incoherent values like e.g. 2018-14 or 2142-03
We decided to not make the 'Date' a mandatory information, so a null value doesn't remove the entire line

In [243]:
df["Date"] = df["Date"].str.replace('[^0-9]', '', regex=True)
df["Date"] = df["Date"].apply(lambda x: str(x))

valid_format = df['Date'].str.match(r'^\d{6}$')
df['Date'] = df['Date'].where(valid_format, '--------')

df["Date"] = df["Date"].apply(lambda x: x[0:4] + '-' + x[4:6])
df["Date"] = df["Date"].str.replace('nan-', '')
df["Date"] = df["Date"].str.replace('-------', '')

df['Date'] = df['Date'].where(pd.to_numeric(df['Date'].str[-2:].astype('str')) <= 12, '')
df['Date'] = df['Date'].where(pd.to_numeric(df['Date'].str[:4].astype('str')) <= 2025, '')


We clean the 'Service' column by correcting the spelling mistakes
We decided to not make the 'Service' a mandatory information, so a null value doesn't remove the entire line

In [244]:
df["Service"] = df["Service"].apply(lambda x: str(x))
df["Service"] = df["Service"].str.replace('nan', '')
df['Service'] = df['Service'].where(df["Service"].str.len() != 8, 'National')
df['Service'] = df['Service'].where(df["Service"].str.len() != 13, 'International')
df['Service'] = df['Service'].where((df["Service"].str.len() == 13) | (df['Service'].str.len() == 8), '')

We clean the 'Departure station' and 'Arrival station' by correcting the spelling mistakes
We decided to make the 'Departure station' and 'Arrival station' mandatory informations, to allow us to locate and verify further informations

In [245]:
df["Departure station"] = df["Departure station"].apply(lambda x: str(x))
df = df.drop(df[df["Departure station"] == 'nan'].index)
df["Arrival station"] = df["Arrival station"].apply(lambda x: str(x))
df = df.drop(df[df["Arrival station"] == 'nan'].index)

In [246]:
for j in range(len(df)):
    station = df.at[df.index[j], 'Departure station']
    
    for element in all_city:
        if len(station) != len(element):
            continue
        if diff(station, element) == 1:
            df.at[df.index[j], 'Departure station'] = 'non valid'
            break

df = df.drop(df[df["Departure station"] == 'non valid'].index)

We convert the columns that represents a number of train to int (because you cannot have an half train) and verify if the values are coherent with the number of trains at departure
We also remove the lines where they are no scheduled trains

In [247]:
df = handle_column_with_train_number('Number of scheduled trains', df)
df = df.drop(df[df["Number of scheduled trains"] == 0].index)
df = handle_column_with_train_number('Number of cancelled trains', df)
df = handle_column_with_train_number('Number of trains delayed at departure', df)
df = handle_column_with_train_number('Number of trains delayed at arrival', df)
df = handle_column_with_train_number('Number of trains delayed > 15min', df)
df = handle_column_with_train_number('Number of trains delayed > 30min', df)
df = handle_column_with_train_number('Number of trains delayed > 60min', df)

We apply the "clean_average" function on all the average columns

In [248]:
df = clean_average('Average journey time', df)
df = clean_average('Average delay of late trains at departure', df)
df = clean_average('Average delay of late trains at arrival', df)
df = clean_average('Average delay of all trains at arrival', df)
df = clean_average('Average delay of trains > 15min (if competing with flights)', df)

We apply the "clean_Pct" function on all the Pct columns

In [249]:
df = clean_Pct("Pct delay due to passenger handling (crowding, disabled persons, connections)", df)
df = clean_Pct('Pct delay due to station management and equipment reuse',df)
df = clean_Pct('Pct delay due to rolling stock', df)
df = clean_Pct('Pct delay due to traffic management', df)
df = clean_Pct('Pct delay due to infrastructure', df)
df = clean_Pct('Pct delay due to external causes', df)

In [250]:
df

,Date,Service,Departure station,Arrival station,Average journey time,Number of scheduled trains,Number of cancelled trains,Cancellation comments,Number of trains delayed at departure,Average delay of late trains at departure,Average delay of all trains at departure,Departure delay comments,Number of trains delayed at arrival,Average delay of late trains at arrival,Average delay of all trains at arrival,Arrival delay comments,Number of trains delayed > 15min,Average delay of trains > 15min (if competing with flights),Number of trains delayed > 30min,Number of trains delayed > 60min,Pct delay due to external causes,Pct delay due to infrastructure,Pct delay due to traffic management,Pct delay due to rolling stock,Pct delay due to station management and equipment reuse,"Pct delay due to passenger handling (crowding, disabled persons, connections)"
1,2018-01,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,2.45.0,222,0,NaN,8,0.2.52,0.10,NaN,34,0.21.31,0.5.41,NaN,22,0.5.41,5,0,0.15.23,0.30.46,0.38.27,0.11.32,0.3.50,0.0.0
2,2018-01,National,PARIS MONTPARNASSE,QUIMPER,3.40.0,248,1,NaN,37,0.9.30,NaN,NaN,0,0.55.41,0.7.34,"Ce mois-ci, l'OD a été touchée par les inciden...",26,0.7.32,17,7,0.26.55,0.38.27,0.0.0,0.19.13,0.0.0,0.0.0
3,2018-01,National,PARIS MONTPARNASSE,ST MALO,2.36.0,102,0,NaN,12,0.19.54,1.97,NaN,13,0.48.37,0.6.47,"Ce mois-ci, l'OD a été touchée par les inciden...",8,0.6.43,6,4,0.23.4,3.38.39,0.7.41,0.15.23,0.7.41,0.0.0
4,2018-01,National,PARIS MONTPARNASSE,ST PIERRE DES CORPS,1.1.0,391,2,NaN,61,0.0.0,0.89,NaN,71,0.12.24,0.3.20,NaN,17,0.3.20,6,0,0.21.12,0.42.25,0.9.5,0.21.12,0.6.3,0.0.0
5,2018-01,National,QUIMPER,PARIS MONTPARNASSE,3.43.0,256,1,NaN,0,0.16.19,0.58,NaN,21,0.39.5,0.5.56,NaN,21,0.5.55,9,0,0.27.46,0.55.33,0.0.0,0.5.33,0.5.33,1.15.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10832,2023-03,National,LYON PART DIEU,PARIS LYON,2.2.0,729,165,NaN,0,0.9.46,4.17,NaN,87,0.55.16,0.9.36,NaN,69,1.6.23,42,30,0.21.50,0.0.0,0.11.29,0.13.47,0.12.38,0.11.29
10833,2022-07,National,PARIS LYON,MONTPELLIER,0.0.0,336,3,NaN,129,0.28.7,10.10,NaN,109,0.52.53,0.19.4,NaN,109,0.52.53,64,28,0.7.20,0.19.15,0.15.35,0.43.7,0.8.15,0.6.25
10835,2020-04,National,PARIS EST,STRASBOURG,0.0.0,40,35,NaN,5,0.1.15,1.25,NaN,3,0.38.27,0.23.58,NaN,3,0.0.0,3,0,0.0.0,0.0.0,0.0.0,0.0.0,0.0.0,0.0.0
10837,2021-03,National,PARIS LYON,VALENCE ALIXAN TGV,2.10.0,178,10,NaN,5,0.2.56,-0.11,NaN,4,1.6.15,0.0.52,NaN,4,1.6.15,4,2,0.25.0,0.25.0,3.18.45,0.25.0,0.0.0,0.25.0


Finally, we convert our cleaned dataframe into a new csv file

In [251]:
df.to_csv('cleaned_dataset.csv')